# 0. 드라이브 연결

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. jsonl 형식 데이터들 load

In [2]:
from datasets import load_dataset

dataset = load_dataset(
    "json",
    data_files={"train": "/content/drive/MyDrive/DILAB/HJ/News_DILAB/news_data/*.jsonl"}   # 구글 드라이브 데이터 경로 지정
)["train"]
print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['title', 'body'],
    num_rows: 312
})


# 2. 불러온 데이터들을 Train/Valid로 분할

In [3]:
# .train_test_split() : 하나의 Dataset을 train용과 test용으로 분리하는 함수
  # test_size = 0.1 : test 사이즈를 10, train 사이즈를 90으로 설정함
  # seed = 42 : shuffle할 시드값을 지정한 것이다. 해당 시드값을 지정해놓으면, 다시 shuffle해도 똑같은 결과값이 나온다.
splits = dataset.train_test_split(test_size=0.1, seed=42)  # 90/10
train_dataset = splits["train"]
valid_dataset = splits["test"]

# 3. train/valid 데이터들 토크나이징

In [4]:
from transformers import AutoTokenizer

# 저장해둔 모델로부터 토크나이저 가져오기
model_path = "/content/drive/MyDrive/DILAB/llama3-Korean-Bllossom-8B"
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 스페셜 토큰 등록, 저장하기
specials = {"additional_special_tokens" : ["<title>", "</title>", "<body>", "</body>"]}
tokenizer.add_special_tokens(specials)
tokenizer.save_pretrained(model_path)

  # to_one_batch() : 제목/본문을 하나의 입력 문자열로 합치는 함수
def to_one_batch(ex):
  return {"text" : "<title>" + ex["title"] + "</title>" + "<body>" + ex["body"] + "</body>"}


# 불러온 데이터들에 to_one_batch 함수 적용하기
  # .map() 함수는 인자로 지정해놓은 함수를 거친 결과를 기존 데이터셋에 추가하는 함수이다.
  # 따라서, "remove_columns"를 지정하지 않으면 해당 데이터셋에 있는 "title"과 "body"가 그대로 남아있으므로, remove_columns를 지정하는 것이다.
  # 이렇게 되면 해당 데이터셋에는 "text" 컬럼의 데이터만 남게 된다.
train_dataset = train_dataset.map(to_one_batch, remove_columns=train_dataset.column_names)
valid_dataset = valid_dataset.map(to_one_batch, remove_columns=valid_dataset.column_names)

  # tokenize_function() : 인자로 들어오는 batch를 토크나이즈 하는 함수
max_length = 2048 # 생성할 토큰 최대 길이
def tokenize_function(batch):
  return tokenizer( # 사전학습된 토크나이저 객체
    batch["text"], # 변환할 원본 텍스트 (문자열 or 문자열 리스트)
    truncation=True, # 문장이 너무 길면 max_length에 맞춰 잘라낼지 여부 결정(절단)
    padding = "max_length", # 문장이 짧은 경우, max_length 길이에 맞춰서 뒤에 [PAD] 토큰 붙여서 길이 맞추기
    max_length=max_length, # 최대 토큰 길이 설정
    return_tensors = "pt" # tokenizer는 파이썬의 dict 형태로 결과를 주는데, 해당 return을 Pytorch Tensor 형태로 반환하도록 설정
  )


train_tokenized = train_dataset.map(tokenize_function, batched=True) # 한번에 여러 샘플 묶음(배치)을 리스트/딕셔너리 형태로 전달하기 위한 설정
valid_tokenized = valid_dataset.map(tokenize_function, batched=True)

print(train_tokenized['text'])



Map:   0%|          | 0/280 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Map:   0%|          | 0/280 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Column(['<title>IT기업들이 몰려 있는 판교 "결국 터질 게 터졌다" 9000명 또 해고한다. 마이크로소프트가 불과 두 달 만에 역대급... "결국 올 게 왔다"며 다른IT기업 직원들도 충격에 휩싸였다. 사실상 인공지능발 구조조정의 신호탄으로 대부분의 회사가 언제든 대규모...</title><body>9000명 또 해고한다. 마이크로소프트가 불과 두 달 만에 역대급 구조조정을 또 발표했다. 앞서 6000명을 해고 한 데 이어 이젠 40~50대 관리자급을 대상으로 한 대규모 인력 감원을 발표했다. 국내 기업들도 50대를 겨냥한 대규모 희망퇴직에 나섰다.\nMS는 최근 약 9000명을 해고할 것이라고 밝혔다. 올해 들어서만 세 번째다. MS는 지난 1월 저성과자 중심으로 전체 직원의 약 1%를 감원했고, 5월에는 6000명을 감원했다. MS 역사상 가장 큰 구조조정이다. ‘\n감원의 주요 타깃은 중간 관리자 직책이다. 40대~50대가 다수인 관리층을 제거해 조직의 유연성과 효율성을 높이기 위한 취지다.\n50대 과장 인력이 수두룩한 KT도 대규모 희망 퇴직을 실시, 2800명이 회사를 떠났다.\nLG유플러스도 파격적인 위로금을 걸고 3년 만에 희망퇴직을 단행한다. 만 50세 이상 10년 이상 근속자가 대상이다. 1968년 이후 출생자의 경우 연봉의 3배까지 받을 수 있다. 최대 4억원대의 위로금과 자녀 학자금까지 지원하는 파격적인 조건이다. SK텔레콤도 만 50세 이상을 대상으로 기존에 주던 5000만원 퇴직위로금을 최대 3억원으로 인상했다. 50대 직원에 대한 희망퇴직 유도와 세대교체를 위해서다.\n앞서 국내 최대 게임업체 엔씨소프트도 약 900명의 희망퇴직을 단행하는 등 구조조정으로 본사 인력을 3000여명이나 줄었다.\n"결국 올 게 왔다"며 다른 IT기업 직원들도 충격에 휩싸였다. 사실상 인공지능발 구조조정의 신호탄으로 대부분의 회사가 언제든 대규모 구조조정을 할 수 있는 상황이기 때문이다.\n특히 경기침체와 AI의 역습으로 고학력, 고임금인 

# 4. 토크나이즈한 값들 드라이브에 저장

In [5]:
train_tokenized.save_to_disk("/content/drive/MyDrive/DILAB/HJ/News_DILAB/train_tokenized")
valid_tokenized.save_to_disk("/content/drive/MyDrive/DILAB/HJ/News_DILAB/valid_tokenized")

Saving the dataset (0/1 shards):   0%|          | 0/280 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/32 [00:00<?, ? examples/s]